In [44]:
import ase
import numpy as np
import cPickle as pck
from ase.visualize import view
import quippy as qp

In [47]:
def qp2ase(qpatoms):
    from ase import Atoms as aseAtoms
    positions = qpatoms.get_positions()
    cell = qpatoms.get_cell()
    numbers = qpatoms.get_atomic_numbers()
    pbc = qpatoms.get_pbc()
    atoms = aseAtoms(numbers=numbers, cell=cell, positions=positions, pbc=pbc)

    for key, item in qpatoms.arrays.iteritems():
        if key in ['positions', 'numbers', 'species', 'map_shift', 'n_neighb']:
            continue
        atoms.set_array(key, item)

    return atoms

def ase2qp(aseatoms):
    from quippy import Atoms as qpAtoms
    positions = aseatoms.get_positions()
    cell = aseatoms.get_cell()
    numbers = aseatoms.get_atomic_numbers()
    pbc = aseatoms.get_pbc()
    return qpAtoms(numbers=numbers,cell=cell,positions=positions,pbc=pbc)


In [2]:
fn = '../structures/partial_input_crystals_sg3-230.pck'
with open(fn,'rb') as f:
    crystals = pck.load(f)

In [20]:
cc = crystals[190][0]
cell = cc.get_cell()
ir_cell = cc.get_reciprocal_cell()
print np.linalg.norm(cell,axis=1)
print cc.get_cell_lengths_and_angles()
print np.linalg.norm(ir_cell,axis=1)
ir_l = np.linalg.norm(ir_cell,axis=1)
dens = 20
nb = np.array(ir_l * dens,dtype=np.int64)
print nb,nb[0]*nb[1]*nb[2] / cc.get_number_of_atoms()


[ 6.08213307  6.08213307  4.44315381]
[   6.08213307    6.08213307    4.44315381   90.           90.          120.        ]
[ 0.18985125  0.18985125  0.22506536]
[3 3 4] 3


In [121]:
cc = unskewCell(crystals[206][0])
cell = cc.get_cell()
ir_cell = cc.get_reciprocal_cell()


ir_l = np.linalg.norm(ir_cell,axis=1)
n_kpt = 1000 / cc.get_number_of_atoms()
n_reg = np.power(n_kpt,1./3.).round()

mid = list(set(range(3)).difference([ir_l.argmin(),ir_l.argmax()]))
ir_l = ir_l / ir_l[mid]

nb = np.array(np.ceil(n_reg*ir_l),dtype=np.int64)
print ir_l
print nb
print cc.get_cell_lengths_and_angles()[0:3]
print nb[0]*nb[1]*nb[2],n_kpt

[ 1.  1.  1.]
[3 3 3]
[ 7.63998959  7.63998959  7.63998959]
27 20.8333333333


In [123]:
def get_kpts(frame,Nkpt=1000):
    ir_cell = frame.get_reciprocal_cell()
    ir_l = np.linalg.norm(ir_cell,axis=1)
    
    n_kpt = float(Nkpt) / frame.get_number_of_atoms()
    n_reg = np.power(n_kpt,1./3.).round()
    # find the midle val to norm with it
    mid = list(set(range(3)).difference([ir_l.argmin(),ir_l.argmax()]))
    ir_l = ir_l / ir_l[mid]
    # get number of k point per directions
    nb = np.array(np.ceil(n_reg*ir_l),dtype=np.int64)
    # makes sure there are at least 3 kpts per directions
    nb[nb <3] = 3
    
    return nb

In [24]:
np.round?

In [86]:
cc = crystals[11][2]
print cc.get_cell_lengths_and_angles()

[   2.22280394    3.53438982    8.74679901   89.99999996  150.81483409
   90.00000007]


In [77]:
view(cc)

In [80]:
dd = ase2qp(cc)
dd.set_cutoff(20.)
dd.unskew_cell()
view(dd)
dd.wrap()
view(dd)
ee = qp2ase(dd)
print ee.get_cell_lengths_and_angles()

[   6.69452234    6.69452234    4.48127216   90.           90.          120.        ]


In [ ]:
cc.set_positions()
cc.set_cell()

In [73]:
np.cos(40*np.pi/180.)

0.76604444311897801

In [83]:
print cc.get_cell_lengths_and_angles()[3:]
np.all(np.abs(np.cos(cc.get_cell_lengths_and_angles()[3:]*np.pi/180.))>=0.5)

[  90.   90.  120.]


False

In [95]:
def isCellSkewed(frame):
    params = np.abs(np.cos(frame.get_cell_lengths_and_angles()[3:]*np.pi/180.))
    return np.any(params>=0.5)
def unskewCell(frame):
    if isCellSkewed(frame):
        dd = ase2qp(cc)
        dd.set_cutoff(20.)
        dd.unskew_cell()
        dd.wrap()
        ee = qp2ase(dd)
        return ee
    else:
        return frame.copy()

In [96]:
cc = crystals[11][2]
view(cc)
ee = unskewCell(cc)
view(ee)